In [ ]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.applications.vgg16 import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data = np.load('drive/MyDrive/Deep Learning Project/Data/CIFAKE_Train.npz')
test_data = np.load('drive/MyDrive/Deep Learning Project/Data/CIFAKE_Test.npz')

In [ ]:
x_train = train_data['images']
y_train = train_data['labels']

In [ ]:
y_train = y_train.astype('int')

In [ ]:
x_test = test_data['images']
y_test = test_data['labels']

In [ ]:
m_train = x_train.shape[0]
x_train = x_train.reshape(m_train, 32 * 32 * 3)

m_test = x_test.shape[0]
x_test = x_test.reshape(m_test, 32 * 32 * 3)

x_train.shape

(100000, 3072)

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

x_train.min(), x_train.max()

(0.0, 1.0)

In [ ]:
x_train = x_train.reshape(m_train, 32, 32, 3)
x_test = x_test.reshape(m_test, 32, 32, 3)
x_train.shape

(100000, 32, 32, 3)

In [ ]:
# uses 10000 photos only
# m = 10000
# x_train = x_train[:m,:]
# y_train = y_train[:m]
# x_test = x_test[:m,:]
# y_test = y_test[:m]

In [ ]:
# df_train = pd.DataFrame({'images': x_train, 'labels': y_train})
# train_images = np.stack(x_train, axis=0)
# m = train_images.shape[0]
# x_train = train_images.reshape(m,32,32,3)
# x_train.shape

In [ ]:
# df_test = pd.DataFrame({'images': x_test, 'labels': y_test})
# test_images = np.stack(x_test, axis=0)
# m = test_images.shape[0]
# x_test = test_images.reshape(m,32,32,3)
# x_test.shape

In [ ]:
# x_train_subset, _, y_train_subset, _ = train_test_split(x_train, y_train, test_size=0.7, random_state=42)

In [ ]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
# model.add(Dense(1,activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 14846273 (56.63 MB)
Trainable params: 14846273 (56.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.layers[0].trainable=False

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

In [ ]:
epochs = 10
batch_size = 32
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
2500/2500 [==============================] - 1298s 519ms/step - loss: 0.4081 - accuracy: 0.8135 - val_loss: 0.3516 - val_accuracy: 0.8435
Epoch 2/10
2500/2500 [==============================] - 1294s 518ms/step - loss: 0.3539 - accuracy: 0.8432 - val_loss: 0.3230 - val_accuracy: 0.8593
Epoch 3/10
2500/2500 [==============================] - 1287s 515ms/step - loss: 0.3345 - accuracy: 0.8542 - val_loss: 0.3084 - val_accuracy: 0.8661
Epoch 4/10
2500/2500 [==============================] - 1285s 514ms/step - loss: 0.3203 - accuracy: 0.8610 - val_loss: 0.2953 - val_accuracy: 0.8744
Epoch 5/10
2500/2500 [==============================] - 1265s 506ms/step - loss: 0.3095 - accuracy: 0.8671 - val_loss: 0.2889 - val_accuracy: 0.8780
Epoch 6/10
2500/2500 [==============================] - 1258s 503ms/step - loss: 0.3028 - accuracy: 0.8719 - val_loss: 0.2828 - val_accuracy: 0.8813
Epoch 7/10
2500/2500 [==============================] - 1267s 507ms/step - loss: 0.2943 - accuracy: 0.8758

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)

625/625 [==============================] - 264s 422ms/step - loss: 0.2950 - accuracy: 0.8760
